In [21]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [22]:
# Extract CSV into DataFrames
interactions_file = 'Resources/RAW_interactions.csv'
interactions_df = pd.read_csv(interactions_file)
interactions_df.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [23]:
# Extract CSV into DataFrames
recipes_file = 'Resources/RAW_recipes.csv'
recipes_df = pd.read_csv(recipes_file)
recipes_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [24]:
# Create a filtered dataframe from specific columns
interactions_cols = ['user_id','recipe_id', 'rating', 'review']
interactions_transformed = interactions_df[interactions_cols]

# Rename the column headers
interactions_transformed = interactions_transformed.rename(columns={
    'user_id': 'user_id',
    'recipe_id': 'recipe_id',
    'rating': 'rating',
    'review': 'review'
    })

# Clean the data by dropping duplicates and setting the index
interactions_transformed.drop_duplicates('recipe_id', inplace=True)
interactions_transformed.set_index('recipe_id', inplace=True)

interactions_transformed.head()

,user_id,rating,review
recipe_id,,,
40893,38094,4,Great with a salad. Cooked on top of stove for...
44394,8937,4,This worked very well and is EASY. I used not...
85009,126440,5,I made the Mexican topping and took it to bunk...
120345,52282,4,very very sweet. after i waited the 2 days i b...
134728,76535,4,Very good!


In [25]:
# Create a filtered dataframe from specific columns
recipes_cols = ['name','id', 'contributor_id', 'description', 'n_ingredients', 'n_steps', 'submitted']
recipes_transformed = recipes_df[recipes_cols]

# Rename the column headers
recipes_transformed = recipes_transformed.rename(columns={
    'name': 'recipe_name',
    'id': 'recipe_id',
    'contributor_id': 'user_id',
    'description': 'description',
    'n_ingredients' : 'n_of_ingredients',
    'n_steps': 'n_of_steps',
    'submitted' : 'date_posted'
    })

# Clean the data by dropping duplicates and setting the index
recipes_transformed.drop_duplicates('recipe_id', inplace=True)
recipes_transformed.set_index('recipe_id', inplace=True)

recipes_transformed.head()

,recipe_name,user_id,description,n_of_ingredients,n_of_steps,date_posted
recipe_id,,,,,,
137739,arriba baked winter squash mexican style,47892,autumn is my favorite time of year to cook! th...,7,11,2005-09-16
31490,a bit different breakfast pizza,26278,this recipe calls for the crust to be prebaked...,6,9,2002-06-17
112140,all in the kitchen chili,196586,this modified version of 'mom's' chili was a h...,13,6,2005-02-25
59389,alouette potatoes,68585,"this is a super easy, great tasting, make ahea...",11,11,2003-04-14
44061,amish tomato ketchup for canning,41706,my dh's amish mother raised him on this recipe...,8,5,2002-10-25


In [26]:
# Create a 2nd filtered dataframe from specific columns to break down ingredients and tags columns into lists
recipe_details_cols = ['id', 'ingredients', 'tags']
recipes_details_transformed = recipes_df[recipe_details_cols]

# Rename the column headers
recipes_details_transformed = recipes_details_transformed.rename(columns={
    'id': 'recipe_id',
    'ingredients': 'ingredients',
    'tags' : 'tags'
    })

# Clean the data by dropping duplicates and setting the index
recipes_details_transformed.drop_duplicates('recipe_id', inplace=True)
recipes_details_transformed.set_index('recipe_id', inplace=True)
recipes_details_transformed.reset_index(level=None, drop=False, inplace=True)
recipes_details_transformed.head()

,recipe_id,ingredients,tags
0,137739,"['winter squash', 'mexican seasoning', 'mixed ...","['60-minutes-or-less', 'time-to-make', 'course..."
1,31490,"['prepared pizza crust', 'sausage patty', 'egg...","['30-minutes-or-less', 'time-to-make', 'course..."
2,112140,"['ground beef', 'yellow onions', 'diced tomato...","['time-to-make', 'course', 'preparation', 'mai..."
3,59389,"['spreadable cheese with garlic and herbs', 'n...","['60-minutes-or-less', 'time-to-make', 'course..."
4,44061,"['tomato juice', 'apple cider vinegar', 'sugar...","['weeknight', 'time-to-make', 'course', 'main-..."


In [27]:
len(recipes_details_transformed.index)

231637

In [28]:
ingredients_df = pd.DataFrame()
tags_df = pd.DataFrame()

recipe_ingredient_id = []
ingredients = []
recipe_tag_id = []
tags = []

rows = len(recipes_details_transformed.index)

for row in range(0,231637):
    recipe = recipes_details_transformed.iloc[row,0]
    for column in range(1,3):
        data = str(recipes_details_transformed.iloc[row,column])
        if data: 
            list = eval(data)
            for item in list:
                if column == 1:
                    recipe_ingredient_id.append(recipe)
                    ingredients.append(item)
                if column == 2:
                    recipe_tag_id.append(recipe)
                    tags.append(item)
ingredients_df['Recipe ID'] = recipe_ingredient_id
ingredients_df['Ingredient'] = ingredients
tags_df['Recipe ID'] = recipe_tag_id
tags_df['Tag'] = tags

In [29]:
ingredients_df.head()

,Recipe ID,Ingredient
0,137739,winter squash
1,137739,mexican seasoning
2,137739,mixed spice
3,137739,honey
4,137739,butter


In [30]:
ingredients_df.to_csv(r'Resources/ingredients.csv')

In [31]:
ingredients_file = 'Resources/ingredients.csv'
ingredients = pd.read_csv(ingredients_file)
ingredients.head()

,Unnamed: 0,Recipe ID,Ingredient
0,0,137739,winter squash
1,1,137739,mexican seasoning
2,2,137739,mixed spice
3,3,137739,honey
4,4,137739,butter


In [33]:
# Create a filtered dataframe from specific columns
ingredients_cols = ['Recipe ID', 'Ingredient']
ingredients_transformed = ingredients[ingredients_cols]

# Rename the column headers
ingredients_transformed = ingredients_transformed.rename(columns={
    'Recipe ID' : 'recipe_id',
    'Ingredient' : 'ingredients'
    })

# Clean the data by dropping duplicates and setting the index
ingredients_transformed.drop_duplicates('recipe_id', inplace=True)
ingredients_transformed.set_index('recipe_id', inplace=True)

ingredients_transformed.head()

,ingredients
recipe_id,
137739,winter squash
31490,prepared pizza crust
112140,ground beef
59389,spreadable cheese with garlic and herbs
44061,tomato juice


In [34]:
tags_df.head()

,Recipe ID,Tag
0,137739,60-minutes-or-less
1,137739,time-to-make
2,137739,course
3,137739,main-ingredient
4,137739,cuisine


In [35]:
tags_df.to_csv(r'Resources/tags.csv')

In [36]:
tags_file = 'Resources/tags.csv'
tags = pd.read_csv(tags_file)
tags.head()

,Unnamed: 0,Recipe ID,Tag
0,0,137739,60-minutes-or-less
1,1,137739,time-to-make
2,2,137739,course
3,3,137739,main-ingredient
4,4,137739,cuisine


In [37]:
# Create a filtered dataframe from specific columns
tags_cols = ['Recipe ID', 'Tag']
tags_transformed = tags[tags_cols]

# Rename the column headers
tags_transformed = tags_transformed.rename(columns={
    'Recipe ID' : 'recipe_id',
    'Tag' : 'tags'
    })

# Clean the data by dropping duplicates and setting the index
tags_transformed.drop_duplicates('recipe_id', inplace=True)
tags_transformed.set_index('recipe_id', inplace=True)

tags_transformed.head()

,tags
recipe_id,
137739,60-minutes-or-less
31490,30-minutes-or-less
112140,time-to-make
59389,60-minutes-or-less
44061,weeknight


In [38]:
connection_string = "postgres:Hercules19!@localhost:5432/recipes_db"
engine = create_engine(f'postgresql://{connection_string}')

In [39]:
# Confirm tables
engine.table_names()

['ingredients',
 'recipes',
 'interactions',
 'tags',
 'ingredients_test',
 'tags_test']

In [ ]:
# Write dataframes into tables in database
recipes_transformed.to_sql(name='recipes', con=engine, if_exists='append', index=True)

In [ ]:
interactions_transformed.to_sql(name='interactions', con=engine, if_exists='append', index=True)

In [40]:
ingredients_transformed.to_sql(name='ingredients_test', con=engine, if_exists='append', index=True)

In [41]:
tags_transformed.to_sql(name='tags_test', con=engine, if_exists='append', index=True)